# Contract Object

In [1]:
import datetime

class Contract:
    def __init__(self, contract_id: int, start_date: datetime.date, end_date: datetime.date, contract_amount: float):
        self.contract_id = contract_id
        self.start_date = start_date
        self.end_date = end_date
        self.contract_amount = contract_amount
        self.duration = self.duration()

    def __repr__(self):
        return f"Contract(contract_id={self.contract_id}, start_date={self.start_date}, end_date={self.end_date}, duration={self.duration} , contract_amount={format(self.contract_amount, ',.2f')})"

    def is_valid(self) -> bool:
        return self.start_date <= datetime.date.today() <= self.end_date

    def duration(self) -> datetime.timedelta:
        return (self.end_date - self.start_date).days

    def time_remaining(self) -> datetime.timedelta:
        if self.is_valid():
            return self.end_date - datetime.date.today()
        return datetime.timedelta(0)

### Create Random Contracts

In [9]:
import random
from typing import List
import json
from IPython.display import display, JSON
import prettyprint

def random_date(start_date: datetime.date, end_date: datetime.date) -> datetime.date:
    days_between = (end_date - start_date).days
    random_days = random.randint(0, days_between)
    return start_date + datetime.timedelta(days=random_days)

def create_random_contracts(num_contracts: int, min_duration: int = 1) -> List[Contract]:
    contracts = []
    start_date = datetime.date(2023, 1, 1)
    end_date = datetime.date(2023, 12, 31)

    for i in range(num_contracts):
        contract_id = i + 1

        # Loop until a valid contract start and end date pair is generated
        while True:
            contract_start_date = random_date(start_date, end_date)
            contract_end_date = random_date(contract_start_date, end_date)
            if (contract_end_date - contract_start_date).days >= min_duration:
                break

        contract_amount = round(random.uniform(1000, 100000), 2)
        contracts.append(Contract(contract_id, contract_start_date, contract_end_date, contract_amount))

    return contracts

def contracts_to_dicts(contracts: List[Contract]) -> list:
    contracts_data = [
        {
            "contract_id": contract.contract_id,
            "start_date": str(contract.start_date),
            "end_date": str(contract.end_date),
            "contract_amount": contract.contract_amount
        }
        for contract in contracts
    ]
    return contracts_data

def find_contract_by_id(contracts: List[Contract], contract_id: int) -> Contract:
    for contract in contracts:
        if contract.contract_id == contract_id:
            return contract
    return None

In [3]:
random_contracts = create_random_contracts(10)
random_contracts

[Contract(contract_id=1, start_date=2023-06-19, end_date=2023-07-02, duration=13 , contract_amount=23,771.41),
 Contract(contract_id=2, start_date=2023-01-09, end_date=2023-04-06, duration=87 , contract_amount=13,479.98),
 Contract(contract_id=3, start_date=2023-03-30, end_date=2023-12-23, duration=268 , contract_amount=33,556.56),
 Contract(contract_id=4, start_date=2023-04-05, end_date=2023-08-15, duration=132 , contract_amount=62,398.69),
 Contract(contract_id=5, start_date=2023-05-11, end_date=2023-08-15, duration=96 , contract_amount=90,757.18),
 Contract(contract_id=6, start_date=2023-03-23, end_date=2023-08-02, duration=132 , contract_amount=64,276.58),
 Contract(contract_id=7, start_date=2023-03-22, end_date=2023-08-26, duration=157 , contract_amount=63,998.16),
 Contract(contract_id=8, start_date=2023-09-11, end_date=2023-10-16, duration=35 , contract_amount=34,148.25),
 Contract(contract_id=9, start_date=2023-07-29, end_date=2023-11-22, duration=116 , contract_amount=97,299.1

## Solution

In [4]:
from typing import List

def contracts_to_accept(contracts: List[Contract]) -> List[int]:
    def can_accept(c1: Contract, c2: Contract) -> bool:
        return c1.end_date < c2.start_date

    contracts.sort(key=lambda c: c.end_date)
    n = len(contracts)
    cas = [c.contract_amount for c in contracts]
    prev = [-1] * n

    for i in range(1, n):
        for j in range(i - 1, -1, -1):
            if can_accept(contracts[j], contracts[i]):
                prev[i] = j
                break

    for i in range(1, n):
        cas[i] = max(cas[i - 1], cas[prev[i]] + contracts[i].contract_amount if prev[i] != -1 else contracts[i].contract_amount)

    result = []
    i = n - 1
    while i >= 0:
        if prev[i] != -1 and cas[i] == cas[prev[i]] + contracts[i].contract_amount:
            result.append(contracts[i].contract_id)
            i = prev[i]
        elif prev[i] == -1 and cas[i] == contracts[i].contract_amount:
            result.append(contracts[i].contract_id)
            break
        else:
            i -= 1

    return result[::-1]


## Testing The Solution

In [6]:
# Create random contracts
random_contracts = create_random_contracts(10)
for c in random_contracts:
    print(c)

Contract(contract_id=1, start_date=2023-11-01, end_date=2023-11-04, duration=3 , contract_amount=20,388.77)
Contract(contract_id=2, start_date=2023-09-18, end_date=2023-11-14, duration=57 , contract_amount=5,724.84)
Contract(contract_id=3, start_date=2023-07-22, end_date=2023-12-11, duration=142 , contract_amount=89,226.04)
Contract(contract_id=4, start_date=2023-02-17, end_date=2023-05-21, duration=93 , contract_amount=83,796.62)
Contract(contract_id=5, start_date=2023-08-14, end_date=2023-12-22, duration=130 , contract_amount=91,371.08)
Contract(contract_id=6, start_date=2023-02-25, end_date=2023-12-17, duration=295 , contract_amount=21,593.70)
Contract(contract_id=7, start_date=2023-07-16, end_date=2023-07-19, duration=3 , contract_amount=80,355.99)
Contract(contract_id=8, start_date=2023-11-30, end_date=2023-12-12, duration=12 , contract_amount=69,846.78)
Contract(contract_id=9, start_date=2023-08-03, end_date=2023-11-18, duration=107 , contract_amount=79,013.82)
Contract(contract_

In [11]:
contracts_to_accept_1 = contracts_to_accept(random_contracts)

total_contract_amount=0
for c_id in contracts_to_accept_1:
    total_contract_amount+= random_contracts[c_id-1].contract_amount
    print(find_contract_by_id(random_contracts, c_id))
print(f"Contract IDs that maximise the gain: {contracts_to_accept_1}, Total contract amount is : {format(total_contract_amount,',.2f')}")

Contract(contract_id=4, start_date=2023-02-17, end_date=2023-05-21, duration=93 , contract_amount=83,796.62)
Contract(contract_id=7, start_date=2023-07-16, end_date=2023-07-19, duration=3 , contract_amount=80,355.99)
Contract(contract_id=9, start_date=2023-08-03, end_date=2023-11-18, duration=107 , contract_amount=79,013.82)
Contract(contract_id=8, start_date=2023-11-30, end_date=2023-12-12, duration=12 , contract_amount=69,846.78)
Contract IDs that maximise the gain: [4, 7, 9, 8], Total contract amount is : 186,391.36


# Part 2

RESTful API

In [14]:
###
# Run this in a different notebook or Python file
###
from flask import Flask, request, jsonify
from typing import List

app = Flask(__name__)

@app.route("/")
def alive():
    return "I am alive!"

@app.route('/api/contracts-to-accept', methods=['POST'])
def api_contracts_to_accept():
    contracts_data = request.json['contracts']
    contracts = [Contract(c['contract_id'], datetime.date.fromisoformat(c['start_date']), datetime.date.fromisoformat(c['end_date']), c['contract_amount']) for c in contracts_data]
    accepted_contract_ids = contracts_to_accept(contracts)
    return jsonify(accepted_contract_ids)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/May/2023 22:15:22] "GET / HTTP/1.1" 200 -


## Test The RESTful API
## Front-End Request Code

In [19]:
import requests

contracts_data = [
    {
        "contract_id": 1,
        "start_date": "2023-05-01",
        "end_date": "2023-05-05",
        "contract_amount": 1000
    },
    {
        "contract_id": 2,
        "start_date": "2023-05-03",
        "end_date": "2023-05-07",
        "contract_amount": 2000
    },
    {
        "contract_id": 3,
        "start_date": "2023-05-06",
        "end_date": "2023-05-10",
        "contract_amount": 3000
    }
]

response = requests.post('http://127.0.0.1:5000/api/contracts-to-accept', json={'contracts': contracts_data})

if response.status_code == 200:
    accepted_contract_ids = response.json()
    print("Accepted contract IDs:", accepted_contract_ids)
else:
    print("Error:", response.status_code, response.text)

Accepted contract IDs: [1, 3]
